# 線形バンディットにおける安全性の保証

参考：
* [Stochastic Bandits with Linear Constraints](https://arxiv.org/abs/2006.10185)

今回は制約を守りながらアームを引く線形バンディットを考えます．
実は制約の設定にも色々あります．
* ナップサックバンディット：予算を使い切らないように，報酬の最大化を目指す
* Conservative バンディット：ベースライン方策の総報酬を下回らないようにアームを引き続ける
* Stage-wise バンディット：上の２つは「アルゴリズムの引いた履歴」に制約が依存することに注意しよう．一方で，Stage-wiseバンディットはその場の制約だけ気にするので，履歴に非依存．

表記：
* アームの集合：$\mathcal{A}$
* アームの引き方の方策$\pi_t$
* 報酬とコスト：$\mathrm{e} r_t=\left\langle x_t, \theta_*\right\rangle+\xi_t^r$ and $c_t=\left\langle x_t, \mu_*\right\rangle+\xi_t^c$
  * ここで，$\theta_* \in \mathbf{R}^d$と$\theta_*\in \mathbf{R}^d$は未知です．
  * 守るべき制約：$\mathbb{E}_{x \sim \pi_t}\left[\left\langle x, \mu_*\right\rangle\right] \leq \tau, \quad \forall t \in[T]$,
* 疑似リグレット：$\mathcal{R}_{\Pi}(T)=\sum_{t=1}^T \mathbb{E}_{x \sim \pi_t^*}\left[\left\langle x, \theta_*\right\rangle\right]-E_{x \sim \pi_t}\left[\left\langle x, \theta_*\right\rangle\right]$
  * 略記：$\mathcal{R}_{\Pi}(T)=\sum_{t=1}^T r_{\pi_t^*}-r_{\pi_t}$
* ノイズの仮定（R-subgaussian）：$\begin{array}{lll}\mathbb{E}\left[\xi_t^r \mid \mathcal{F}_{t-1}\right]=0, & \mathbb{E}\left[\exp \left(\alpha \xi_t^r\right) \mid \mathcal{F}_{t-1}\right] \leq \exp \left(\alpha^2 R^2 / 2\right), & \forall \alpha \in \mathbb{R}, \\ \mathbb{E}\left[\xi_t^c \mid \mathcal{F}_{t-1}\right]=0, & \mathbb{E}\left[\exp \left(\alpha \xi_t^c\right) \mid \mathcal{F}_{t-1}\right] \leq \exp \left(\alpha^2 R^2 / 2\right), & \forall \alpha \in \mathbb{R},\end{array}$
* $\left\|\theta_*\right\| \leq S$ and $\left\|\mu_*\right\| \leq S^2$ を満たす定数$S$が既知
* $\max _{t \in[T]} \max _{x \in \mathcal{A}_t}\|x\| \leq L$.
* 報酬は有界：$\left\langle x, \theta_*\right\rangle \in[0, 1]$かつ$\left\langle x, \mu_*\right\rangle \in [0, 1]$
* 安全な行動$x_0$とそのコストが既知：$\langle x_0, \mu_*\rangle = c_0 < \tau$
  * 正規化された安全方策：$e_0:=x_0 /\left\|x_0\right\|$
  * 安全方策のスパン：$\mathcal{V}_o:=\operatorname{span}\left(x_0\right)=\left\{\eta x_0: \eta \in \mathbb{R}\right\}$
  * その核空間：$\mathcal{V}_o^{\perp}=\left\{x \in \mathbb{R}^d:\langle x, y\rangle=0, \forall y \in \mathcal{V}_o\right\}$
  * $\mathcal{V}_o$への$x$の射影：$x^o:=\left\langle x, e_0\right\rangle e_0$
  * $\mathcal{V}^\perp_o$への$x$の射影：$x^{o, \perp}:=x - x^o$

## Optimistic-Pessimistic Linear Banditアルゴリズム

1. $\theta_*$を推定します：$\widehat{\theta}_t=\Sigma_t^{-1} \sum_{s=1}^{t-1} r_s x_s$
   * ここで，$\Sigma_t=\lambda I+\sum_{s=1}^{t-1} x_s x_s^{\top}$はグラム行列です．
2. $\mu_*$の核空間への射影を推定します：$\widehat{\mu}_t^{o, \perp}=\left(\Sigma_t^{o, \perp}\right)^{-1} \sum_{s=1}^{t-1} c_s^{o, \perp} x_s^{o, \perp}$
   * ここで，$\Sigma_t^{o, \perp}=\lambda I_{\mathcal{V}_{\circ}^{\perp}}+\sum_{s=1}^{t-1} x_s^{o, \perp}\left(x_s^{o, \perp}\right)^{\top}$
   * $c_t^{o, \perp}=c_t-\frac{\left\langle x_t, e_0\right\rangle}{\left\|x_0\right\|} c_0$（これの導出は省略）
   * $I_{\mathcal{V}_o^{\perp}}=I_{d \times d}-\frac{1}{\left\|x_0\right\|^2} x_0 x_0^{\top}$.
   * ここで注意として，$\Sigma_t^{o, \perp}$は階数不足の可能性があります．その場合は疑似逆行列を取ることにします．
3. 信頼集合を計算します：
$
\mathcal{C}_t^r\left(\alpha_r\right)=\left\{\theta \in \mathbb{R}^d:\left\|\theta-\widehat{\theta}_t\right\|_{\Sigma_t} \leq \alpha_r \beta_t(\delta, d)\right\}, \quad \mathcal{C}_t^c\left(\alpha_c\right)=\left\{\mu \in \mathcal{V}_o^{\perp}:\left\|\mu-\widehat{\mu}_t^{o, \perp}\right\|_{\Sigma_t^{o, \perp}} \leq \alpha_c \beta_t(\delta, d-1)\right\},
$
   * ここで，$\beta_t(\delta, d)=R \sqrt{d \log \left(\frac{1+(t-1) L^2 / \lambda}{\delta}\right)}+\sqrt{\lambda} S$です．これはYadkoriの信頼半径を使って設計してます．[BANDIT_General_linear_improved](BANDIT_General_linear_improved.ipynb)参照．
4. Optimisticな報酬とPessimisticなコストを計算します：$\widetilde{r}_{\pi, t}:=\max _{\theta \in \mathcal{C}_t^r\left(\alpha_r\right)} \mathbb{E}_{x \sim \pi}[\langle x, \theta\rangle], \quad \widetilde{c}_{\pi, t}:=\frac{\left\langle x_\pi^o, e_0\right\rangle c_0}{\left\|x_0\right\|}+\max _{\mu \in \mathcal{C}_t^c\left(\alpha_c\right)} \mathbb{E}_{x \sim \pi}[\langle x, \mu\rangle]$
   * これは解析解が書けます（証明と表記略．Proposition 1を見てね）
5. 安全な方策の集合を構築します：$\Pi_t=\left\{\pi \in \Delta_{\mathcal{A}_t}: \widetilde{c}_{\pi, t} \leq \tau\right\}$
   * 実は，$\Pi_t$には確率１で$x_0$が入っていることが言えます．なぜなら，
   $x_{\pi_0}^o=x_0, x_{\pi_0}^{o, \perp}=0$, and $\frac{\left\langle x_{\pi_0}^o, e_0\right\rangle c_0}{\left\|x_0\right\|}=c_0$であり，さらに$C^c_t$は$\mu\in \mathcal{V}_o^\perp$の部分集合なので，Pessimisticの２項目が零になります．
   * また，$\Pi_t$に入っているすべての方策は確率$1-\delta$以上でFeasibleであることも言えます．
6. あとは貪欲に行動を選択します


**注意**

**このアルゴリズムの効率良い計算方法は与えられてないっぽいです．**
実際，
$$
\begin{aligned}
\max _{\pi \in \Delta}\left\langle\mathcal{A}_t\right. & \left\langle x_\pi, \widehat{\theta}_t\right\rangle+\alpha_r \beta_t(\delta, d)\left\|x_\pi\right\|_{\Sigma_t^{-1}} \\
\quad \text { s.t. } & \frac{\left\langle x_\pi^o, e_0\right\rangle c_0}{\left\|x_0\right\|}+\left\langle x_\pi^{o, \perp}, \widehat{\mu}_t^{o, \perp}\right\rangle+\alpha_c \beta_t(\delta, d-1)\left\|x_\pi^{o, \perp}\right\|_{\left(\Sigma_t^{o, \perp}\right)^{-1}} \leq \tau .
\end{aligned}
$$
を解く必要がありますが，安全な方策の集合$\Pi_t$が計算できないっぽいですね．

## 別の論文

* [Safe Linear Thompson Sampling With Side Information](https://ieeexplore.ieee.org/abstract/document/9457159?casa_token=jVBRjCJq8qUAAAAA:tAiIIRDNw8UreUJQhszji5fboRk-qFxRVJf7Q9NHxFQl2okLvIszZl5sVFw-jtzGA2jReFrFuAfX)：Thompson samplingを考えてます．今回はパス．
* [Linear Stochastic Bandits Under Safety Constraints](https://arxiv.org/abs/1908.05814)

